In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import warnings
import ast
warnings.filterwarnings('ignore')

In [2]:
# Cargar los datos de juegos (descripciones)
df_juegos = pd.read_csv('../data_extraction/boardgames_10000_juegos.csv')

In [3]:
df_juegos.columns

Index(['BGGId', 'Name', 'Year_Published', 'Description', 'Min_Players',
       'Max_Players', 'Min_Playtime', 'Max_Playtime', 'Average_Rating',
       'Bayesian_Average_Rating', 'Number_of_Ratings', 'Mechanics',
       'Categories'],
      dtype='object')

In [4]:
# Obtener la descripción de los juegos
df_descripciones = df_juegos[['BGGId','Name','Description', 'Mechanics', 'Categories', 'Average_Rating', 'Bayesian_Average_Rating']]  

In [5]:
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    10000 non-null  int64  
 1   Name                     10000 non-null  object 
 2   Description              9992 non-null   object 
 3   Mechanics                10000 non-null  object 
 4   Categories               10000 non-null  object 
 5   Average_Rating           10000 non-null  float64
 6   Bayesian_Average_Rating  10000 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 547.0+ KB


In [6]:
df_descripciones.dropna(inplace=True)
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    9992 non-null   int64  
 1   Name                     9992 non-null   object 
 2   Description              9992 non-null   object 
 3   Mechanics                9992 non-null   object 
 4   Categories               9992 non-null   object 
 5   Average_Rating           9992 non-null   float64
 6   Bayesian_Average_Rating  9992 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 624.5+ KB


In [7]:
df_descripciones.sample(5)

,BGGId,Name,Description,Mechanics,Categories,Average_Rating,Bayesian_Average_Rating
6591,29573,Winnie the Pooh Honey Tree Game,"From the box:&#10;&#10;""The bees have left the...","['Modular Board', 'Point to Point Movement']","['Animals', ""Children's Game"", 'Movies / TV / ...",4.5,0.0
3742,288680,EFMB: Endless Forms Most Beautiful,"Imps exist all around us, just beyond the limi...",[],[],0.0,0.0
357,30368,"""Struggle""...a game about life",Struggle is a game about life. Each player/tea...,['Dice Rolling'],"['Dice', 'Economic', 'Trivia']",3.5,0.0
538,60356,Purgatorium: Seasons of the Soul,"When a character dies, it is a monumental even...",[],[],5.0,0.0
9607,245286,Flash Gordon Archetypes,Publisher's blurb:&#10;&#10;Get to the action ...,[],[],7.0,0.0


In [8]:
# Asegúrate de descargar stopwords y wordnet de nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Función mejorada de preprocesamiento de texto
def preprocesar_descripcion(texto):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    # Eliminar entidades HTML y otros caracteres no deseados
    texto = re.sub(r'&#?\w+;', ' ', texto)  # Reemplazar entidades HTML como &#10; 
    texto = re.sub(r'\W+', ' ', texto)  # Eliminar caracteres no alfanuméricos
    texto = re.sub(r'\d+', '', texto)  # Eliminar números
    
    # Convertir a minúsculas, eliminar stopwords y lematizar (solo si el texto es válido)
    if isinstance(texto, str):
        palabras = [lemmatizer.lemmatize(palabra) for palabra in texto.lower().split() if palabra not in stop_words]
        return " ".join(palabras)
    else:
        return ""  # Devolver cadena vacía si no es un texto válido



[nltk_data] Downloading package stopwords to C:\Users\Laura
[nltk_data]     Ortiz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Laura
[nltk_data]     Ortiz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# Aplicar el preprocesamiento mejorado a las descripciones
df_descripciones['Descripcion_completa'] = df_descripciones['Description'].apply(preprocesar_descripcion)

In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Entrenar Doc2Vec usando los índices del DataFrame como etiquetas
documents = [TaggedDocument(doc.split(), [idx]) for idx, doc in zip(df_descripciones.index, df_descripciones['Descripcion_completa'])]




In [11]:
# Entrenar el modelo Doc2Vec
model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4, seed = 42)

In [12]:
# Función para recomendar juegos usando Doc2Vec y similaridad del coseno
def recomendar_juegos_doc2vec(prompt, df_descripciones, model, top_n=5):
    # Preprocesar el prompt y vectorizarlo usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Obtener los vectores de todos los juegos
    vectors = [model.dv[i] for i in range(len(df_descripciones))]

    # Calcular la similaridad del coseno entre el prompt y los juegos
    cosine_similarities = cosine_similarity([prompt_vector], vectors).flatten()

    # Obtener los índices de los juegos con mayor similaridad
    indices_recomendados = cosine_similarities.argsort()[-top_n:][::-1]

    # Mostrar los juegos recomendados
    print("Juegos recomendados:")
    for index in indices_recomendados:
        print(f"{df_descripciones.iloc[index]['Name']} (Similaridad: {cosine_similarities[index]:.4f})")

In [13]:
# Prompt del usuario
prompt_usuario = 'I am looking for a game that is great for solo play and lasts about 30 minutes. I love games with a rich storytelling experience and immersive themes.'
# Ejecutar la recomendación usando Doc2Vec
recomendar_juegos_doc2vec(prompt_usuario, df_descripciones, model, top_n=5)

Juegos recomendados:
Trapped in Human Town (Similaridad: 0.7339)
Justice Online Treacherous Waters (Similaridad: 0.7297)
Questions Without Answers (Similaridad: 0.7140)
Tales From Summerset Bay - In the Crosshairs (Similaridad: 0.7070)
Echelon (Similaridad: 0.7005)


In [14]:
def recomendar_juegos_doc2vec_con_valoraciones(prompt, df_descripciones, model, top_n=5, umbral_similaridad=0.1):
    # Preprocesar el prompt y vectorizarlo usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Obtener los vectores de todos los juegos usando los índices correctos del DataFrame
    vectors = [model.dv[idx] for idx in df_descripciones.index]

    # Calcular la similaridad del coseno entre el prompt y los juegos
    cosine_similarities = cosine_similarity([prompt_vector], vectors).flatten()

    # Asegurarse de que las valoraciones coincidan con los juegos vectorizados
    valoraciones = df_descripciones.loc[df_descripciones.index.isin(df_descripciones.index), 'Average_Rating'].fillna(0).values

    # Asegurar que los tamaños de cosine_similarities y valoraciones coincidan
    if len(cosine_similarities) != len(valoraciones):
        min_len = min(len(cosine_similarities), len(valoraciones))
        cosine_similarities = cosine_similarities[:min_len]
        valoraciones = valoraciones[:min_len]

    # Crear una métrica combinada (similaridad * valoraciones)
    similaridad_ponderada = cosine_similarities * valoraciones

    # Filtrar recomendaciones con similaridad negativa o muy baja
    indices_validos = [i for i, sim in enumerate(cosine_similarities) if sim > umbral_similaridad]

    # Ordenar los juegos con mayor similaridad ponderada
    indices_recomendados = sorted(indices_validos, key=lambda i: similaridad_ponderada[i], reverse=True)[:top_n]

    # Mostrar los juegos recomendados
    print("Juegos recomendados (con valoraciones y similaridad del coseno):")
    for index in indices_recomendados:
        print(f"{df_descripciones.iloc[index]['Name']} (Similaridad ponderada: {similaridad_ponderada[index]:.4f}, "
              f"Similaridad del coseno: {cosine_similarities[index]:.4f}, "
              f"Valoración: {valoraciones[index]:.2f})")



In [15]:
prompt_usuario = 'I am looking for a game that is great for solo play and lasts about 30 minutes. I love games with a rich storytelling experience and immersive themes.'

In [16]:
# Ejecutar la recomendación usando Doc2Vec con valoraciones ponderadas y umbral
recomendar_juegos_doc2vec_con_valoraciones(prompt_usuario, df_descripciones, model, top_n=5, umbral_similaridad=0.1)

Juegos recomendados (con valoraciones y similaridad del coseno):
Punctuation Pirates and Their Grammar Galleons (Similaridad ponderada: 7.3616, Similaridad del coseno: 0.7362, Valoración: 10.00)
Forever Young (Similaridad ponderada: 6.6548, Similaridad del coseno: 0.6655, Valoración: 10.00)
missed messages (Similaridad ponderada: 6.1914, Similaridad del coseno: 0.7430, Valoración: 8.33)
Retro Rampage Trivia Quest (Similaridad ponderada: 6.1039, Similaridad del coseno: 0.6104, Valoración: 10.00)
Kinect Sesame Street TV (Similaridad ponderada: 5.9661, Similaridad del coseno: 0.7458, Valoración: 8.00)


In [19]:
def recomendar_juegos_doc2vec_con_valoraciones(prompt, df_descripciones, model, top_n=5, umbral_similaridad=0.1, w=0.7):
    # Preprocesar el prompt y vectorizarlo usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Obtener los vectores de todos los juegos usando los índices correctos del DataFrame
    vectors = [model.dv[idx] for idx in df_descripciones.index]

    # Calcular la similaridad del coseno entre el prompt y los juegos
    cosine_similarities = cosine_similarity([prompt_vector], vectors).flatten()

    # Asegurarse de que las valoraciones coincidan con los juegos vectorizados
    valoraciones = df_descripciones.loc[df_descripciones.index.isin(df_descripciones.index), 'Average_Rating'].fillna(0).values

    # Normalizar las valoraciones a una escala de 0 a 1
    valoraciones_normalizadas = valoraciones / 10  # Suponiendo que la escala máxima es 10

    # Asegurar que los tamaños de cosine_similarities y valoraciones coincidan
    if len(cosine_similarities) != len(valoraciones):
        min_len = min(len(cosine_similarities), len(valoraciones))
        cosine_similarities = cosine_similarities[:min_len]
        valoraciones_normalizadas = valoraciones_normalizadas[:min_len]

    # Crear una métrica ponderada que priorice más la similaridad del coseno
    similaridad_ponderada = (cosine_similarities * w) + (valoraciones_normalizadas * (1 - w))

    # Filtrar recomendaciones con similaridad negativa o muy baja
    indices_validos = [i for i, sim in enumerate(cosine_similarities) if sim > umbral_similaridad]

    # Ordenar los juegos con mayor similaridad ponderada
    indices_recomendados = sorted(indices_validos, key=lambda i: similaridad_ponderada[i], reverse=True)[:top_n]

    # Mostrar los juegos recomendados
    print(f"Juegos recomendados (priorizando similaridad del coseno):")
    for index in indices_recomendados:
        print(f"{df_descripciones.iloc[index]['Name']} (Similaridad ponderada: {similaridad_ponderada[index]:.4f}, "
              f"Similaridad del coseno: {cosine_similarities[index]:.4f}, "
              f"Valoración: {valoraciones[index]:.2f})")


In [20]:
# Ejecutar la recomendación priorizando la similaridad del coseno (peso w = 0.7)
recomendar_juegos_doc2vec_con_valoraciones(prompt_usuario, df_descripciones, model, top_n=5, umbral_similaridad=0.1, w=0.7)

Juegos recomendados (priorizando similaridad del coseno):
Punctuation Pirates and Their Grammar Galleons (Similaridad ponderada: 0.7979, Similaridad del coseno: 0.7113, Valoración: 10.00)
missed messages (Similaridad ponderada: 0.7683, Similaridad del coseno: 0.7405, Valoración: 8.33)
Forever Young (Similaridad ponderada: 0.7384, Similaridad del coseno: 0.6263, Valoración: 10.00)
Holiday Heroes (Similaridad ponderada: 0.7383, Similaridad del coseno: 0.7119, Valoración: 8.00)
Kinect Sesame Street TV (Similaridad ponderada: 0.7149, Similaridad del coseno: 0.6785, Valoración: 8.00)
